Let $G$ be a finite group. A representation $(\rho, V)$ of $G$ consists of a finite-dimensional complex vector space $V$ and a group homomorphism $\rho: G \to GL(V)$. If $V$ is $m$-dimensional then we may identify $GL(V)$ with $GL(m, \mathbb{C})$, the group of $m \times m$ invertible matrices over $\mathbb{C}$. The character of $\rho$ is the function $\chi: G \to \mathbb{C}$ given by $\chi(g) = \mathop{tr}\rho(g)$. It is known that representations are uniquely determined (up to equivalence) by their characters.

Let $G$ have conjugacy classes $C_1, \dots, C_r$. The character table of $G$ is the $r \times r$ complex matrix with entries $\chi_i(g_j)$ where $\chi_1, \dots, \chi_r$ are the irreducible characters of $G$ and $g_1, \dots, g_r$ are representatives for the conjugacy classes. The character table conveys a great deal of information about the group $G$. For example, it can be used to decompose any given character as a sum of irreducibles, or to find the normal subgroups of $G$.

A permutation $\pi$ of $X = \{1, \dots, n\}$ is a bijective function from $X \to X$. If $x$ is an element of $X$ then the image of $x$ under $\pi$ is written $\pi x$. If $\pi_1$ and $\pi_2$ are permutations then their product
$\pi_1 \cdot \pi_2$ maps $x \mapsto \pi_1(\pi_2x)$. The set of all permutations of $X$ is the symmetric group $S_n$. A permutation group is a subgroup of $S_n$ for some $n$. We specify a permutation group by giving a (usually very small) set of generating permutations $\pi_1, \dots, \pi_t$.

In [1]:
import collections
import numpy as np

class Permutation:
    '''
    A class to represent a permutation on the set {0, 1, ..., n-1}.
    The permutation is stored as a tuple representing its mapping.
    '''

    def __init__(self, mapping):
        '''
        Initializes the Permutation object.
        Args:
            mapping: A sequence representing the permutation's mapping.
            It must be a permutation of (0, 1, ..., n-1).
        '''
        self.mapping = tuple(mapping)
        self.size = len(mapping)

    def __mul__(self, other):
        '''
        Composes this permutation with another (self * other).
        '''
        if self.size != other.size:
            raise ValueError("Permutations must be of the same size to compose.")

        # The new mapping is created by applying self to the result of other's mapping
        new_mapping = [self.mapping[other.mapping[i]] for i in range(self.size)]
        return Permutation(new_mapping)

    def inverse(self):
        '''
        Computes the inverse of the permutation.
        '''
        inv_mapping = [0] * self.size
        for i in range(self.size):
            inv_mapping[self.mapping[i]] = i
        return Permutation(inv_mapping)

    def __eq__(self, other):
        '''
        Checks for equality between two permutation objects.
        '''
        return self.mapping == other.mapping

    def __hash__(self):
        '''
        Computes the hash of the permutation mapping.
        '''
        return hash(self.mapping)

    def __repr__(self):
        '''
        Returns a string representation of the permutation.
        '''
        return f"Perm{self.mapping}"

    def __lt__(self, other):
        return self.mapping < other.mapping

In [2]:
def generate_group(generators):
    '''
    Generates all elements of a permutation group from a set of generators
    using a breadth-first search on the group's Cayley graph.
    Args:
        generators: A list of permutations that generate the group.
    Returns:
        set of Permutation: A set containing all elements of the group.
    '''
    if not generators:
        return set()

    n = generators[0].size
    identity = Permutation(tuple(range(n)))

    group_elements = {identity}
    # A worklist for elements whose neighbors we need to explore
    worklist = [identity]

    head = 0
    while head < len(worklist):
        current_element = worklist[head]
        head += 1

        # Explore new elements by multiplying with each generator
        for gen in generators:
            new_element = current_element * gen
            if new_element not in group_elements:
                group_elements.add(new_element)
                worklist.append(new_element)

    return group_elements

def find_conjugacy_classes(group_elements):
    '''
    Partitions the elements of a group into its conjugacy classes.
    Args:
        group_elements: All elements of the group.
    Returns:
        A list where each element is a set representing one conjugacy class.
    '''
    if not group_elements:
        return []

    conjugacy_classes = []
    processed_elements = set()
    sorted_elements = sorted(list(group_elements))

    for g in group_elements:
        if g in processed_elements:
            continue

        current_class = set()
        for h in group_elements:
            h_inv = h.inverse()
            conjugate = h * g * h_inv
            current_class.add(conjugate)

        conjugacy_classes.append(current_class)
        # Add all elements of the newly found class to the processed set
        # to avoid redundant calculations.
        processed_elements.update(current_class)

    return conjugacy_classes

print("Computing conjugacy classes for G = S3.")
# S3 is generated by a transposition and a 3-cycle.
gen_transposition = Permutation((1, 0, 2))
gen_3_cycle = Permutation((1, 2, 0))
s3_generators = [gen_transposition, gen_3_cycle]

group = generate_group(s3_generators)
classes = find_conjugacy_classes(group)
classes.sort(key=len)
print(f"Group Order: {len(group)}")

print("\nConjugacy classes")
for i, c_class in enumerate(classes):
    representative = next(iter(c_class))
    print(f"Class {i+1}:")
    print(f"  Size: {len(c_class)}")
    print(f"  Representative: {representative}")

Computing conjugacy classes for G = S3.
Group Order: 6

Conjugacy classes
Class 1:
  Size: 1
  Representative: Perm(0, 1, 2)
Class 2:
  Size: 2
  Representative: Perm(2, 0, 1)
Class 3:
  Size: 3
  Representative: Perm(2, 1, 0)


Let $|G|$ be the order of the group, $n$ be the degree of the permutations and $t$ be the number of generators.

1.  Permutation operations:
    *   Storing a permutation requires $O(n)$ space.
    *   Multiplication and inversion of permutations both take $O(n)$ time.

2.  Group generation:
    *   This function explores the group structure starting from the identity.
    *   For each of the $|G|$ elements, it performs $t$ multiplications.
    *   Each multiplication costs $O(n)$, and checking for existence in a hash set also takes $O(n)$.
    *   Thus, the total complexity is $O(|G|tn)$.
3. Conjugacy class search:
    *   The algorithm iterates through each element $g$ of $G$.
    *   For each new $g$ (one per class), it forms the class by computing $hgh^{-1}$ for all $h \in G$.
    *   This involves $|G|$ iterations, each performing an inversion and two multiplications, costing $O(n)$.
    *   The worst-case complexity is when we compute a class for a representative of each class.
    *   This leads to an overall total time complexity of $O(|G|^2n)$.

A formal sum $\sum_{g \in G} \lambda_g g$, where $\lambda_g \in \mathbb{C}$, belongs to the centre $Z(\mathbb{C}[G])$ of the group ring $\mathbb{C}[G]$ if and only if the function $g \mapsto \lambda_g$ is constant on conjugacy classes. Thus $Z(\mathbb{C}[G])$ is a complex vector space with basis $b_1, \dots, b_r$ where $b_i = \sum_{g \in C_i} g$. Write
\begin{equation}
    b_ib_j = \sum_{k=1}^r \nu_{ijk}b_k
\end{equation}
for all $1 \leq i, j \leq r$. Define a matrix $N_i = (\nu_{ijk})_{j,k}$. We claim that $N_1, \dots, N_r$ pairwise commute.

Recall that the centre of a group is the set of elements that commute with every element in the group. The basis elements $b_i$, which are sums over conjugacy classes, form a basis for $Z(\mathbb{C}[G])$. Since $b_i, b_j \in Z(\mathbb{C}[G])$, then $b_ib_j \in Z(\mathbb{C}[G])$, which proves the inheritence of a commutative group structure in the centre.

Consider the product $b_ib_jb_l$. We can expand this in two different ways. Computing the left product first,
\begin{equation}
    (b_i b_j)b_l = \left(\sum_k \nu_{ijk} b_k\right) b_l = \sum_k \nu_{ijk} (b_k b_l) = \sum_k \nu_ijk \left(\sum_m \nu_{klm} b_m\right) = \sum_m \left(\sum_k \nu_{ijk} \nu_{klm}\right) b_m.
\end{equation}
The coefficient of $b_m$ in this expansion is $\sum_k \nu_{ijk}\nu_{klm}$, which is is precisely the $(j, m)$-th entry of the matrix product $N_i N_l$. Now computing the right product first,
\begin{equation}
    b_i (b_j b_l) = b_i\left(\sum_k \nu_{jlk} b_k\right) = \sum_k \nu_{jlk} (b_i b_k) = \sum_k \nu_{jlk} \left(\sum_m \nu_{ikm} b_m\right) = \sum_m \left(\sum_k \nu_{jlk} \nu_{ikm}\right) b_m.
\end{equation}
The coefficient of b_m here is $\sum_k \nu{_jlk}\nu_{ikm}$. This is the $(l, m)$-th entry of $N_j$ multiplied by the $(k, m)$-th entry of $N_i$, summed over k. Let $(N_i)_{km} = \nu_{ikm}$ and $(N_j)_{lk} = \nu_{jlk}$. The coefficient is $\sum_k (N_j)_{lk} (N_i)_{km}$ and is the $(l, m)$-th entry of the matrix product $N_j N_i$.

Since the basis elements $b_m$ are linearly independent, the coefficients from the two different expansions must be equal. Therefore, for all $j, l, m$, we have that the $(l, m)$-th entry of $N_iN_j$ is $\sum_k \nu_{ijk} \nu_{jkm}$ and the (l, m)-th entry of $N_jN_i$ is $\sum_k \nu_{jik} \nu_{ikm}$. By commutativity of the centre, $b_ib_j = b_jb_i$, we have
\begin{equation}
    \sum_k \nu_{ijk} b_k = \sum_k \nu_{jik} b_k,
\end{equation}
which implies $\nu_{ijk} = \nu_{jik}$ for all $i, j, k$. Using associativity,
\begin{equation}
    \sum_k \nu_{ijk} \nu_{klm} = \sum_k \nu_{ilk} \nu_{kjm}.
\end{equation}
This corresponds to $(N_iN_l)_{jm} = (N_lN_i)_{jm}$. which holds for all entries $(j, m)$, proving that $N_iN_l = N_lN_i$ for all $1 \leq i, l \leq r$.


In [3]:
def compute_nu_ijk(classes):
    '''
    Computes the integers v_ijk such that b_i * b_j = sum_k(v_ijk b_k).
    Args:
        classes: The conjugacy classes of the group.
    Returns:
        A 3D array containing the v_ijk values.
    '''
    r = len(classes)
    nu = np.zeros((r, r, r), dtype=int)

    # Create a map from each group element to the index of its class
    element_to_class_idx = {}
    for k, class_k in enumerate(classes):
        for g in class_k:
            element_to_class_idx[g] = k

    for i in range(r):
        for j in range(r):
            # Compute the product b_i * b_j by multiplying every element in
            # class i with every element in class j.
            product_counts = collections.defaultdict(int)
            for g1 in classes[i]:
                for g2 in classes[j]:
                    product = g1 * g2
                    # The coefficient of each element in the sum is tallied.
                    product_counts[product] += 1

            # Decompose the product sum into a linear combination of class sums b_k
            for product, count in product_counts.items():
                k = element_to_class_idx[product]
                # The structure constants v_ijk are coefficients in this decomposition.
                # Since all elements in a class sum have the same coefficient,
                # we can set v_ijk to this count.
                nu[i, j, k] = count
    return nu

# Generators for S3
gen_transposition = Permutation((1, 0, 2))
gen_3_cycle = Permutation((1, 2, 0))
s3_generators = [gen_transposition, gen_3_cycle]
group = generate_group(s3_generators)
classes = find_conjugacy_classes(group)
r = len(classes)

print("Conjugacy classes of S3 (basis elements b_i):")
for i, c in enumerate(classes):
    rep = sorted(list(c))[0]
    print(f"  b_{i+1}: Class of {rep}, Size = {len(c)}")

nu = compute_nu_ijk(classes)

# Construct the matrices N_i
matrices_N = []
for i in range(r):
    # N_i is the matrix with (j,k)-entry v_ijk
    matrices_N.append(nu[i, :, :])

print("\nMatrices N_i:")
for i, N in enumerate(matrices_N):
    print(f"N_{i+1} = \n{N}")

# Verify that the matrices pairwise commute
print("\nVerifying commutative property (N_i * N_j = N_j * N_i):")
for i in range(r):
    for j in range(i + 1, r):
        N_i = matrices_N[i]
        N_j = matrices_N[j]

        product1 = np.dot(N_i, N_j)
        product2 = np.dot(N_j, N_i)

        print(f"Comparing N_{i+1} * N_{j+1} and N_{j+1} * N_{i+1}:")
        if not np.array_equal(product1, product2):
            print("  -> Matrices do NOT commute!")
        else:
            print("  -> Matrices commute.")

Conjugacy classes of S3 (basis elements b_i):
  b_1: Class of Perm(0, 2, 1), Size = 3
  b_2: Class of Perm(1, 2, 0), Size = 2
  b_3: Class of Perm(0, 1, 2), Size = 1

Matrices N_i:
N_1 = 
[[0 3 3]
 [2 0 0]
 [1 0 0]]
N_2 = 
[[2 0 0]
 [0 1 2]
 [0 1 0]]
N_3 = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]

Verifying commutative property (N_i * N_j = N_j * N_i):
Comparing N_1 * N_2 and N_2 * N_1:
  -> Matrices commute.
Comparing N_1 * N_3 and N_3 * N_1:
  -> Matrices commute.
Comparing N_2 * N_3 and N_3 * N_2:
  -> Matrices commute.
